In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import OneHotEncoder, StandardScaler
#from sklearn.compose import ColumnTransformer
#from sklearn.pipeline import Pipeline
#from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report, confusion_matrix

In [2]:
app = pd.read_csv('Data/application_record.csv')
credit = pd.read_csv('Data/credit_record.csv')

In [3]:
app.head(5)
app.dtypes

ID                       int64
CODE_GENDER             object
FLAG_OWN_CAR            object
FLAG_OWN_REALTY         object
CNT_CHILDREN             int64
AMT_INCOME_TOTAL       float64
NAME_INCOME_TYPE        object
NAME_EDUCATION_TYPE     object
NAME_FAMILY_STATUS      object
NAME_HOUSING_TYPE       object
DAYS_BIRTH               int64
DAYS_EMPLOYED            int64
FLAG_MOBIL               int64
FLAG_WORK_PHONE          int64
FLAG_PHONE               int64
FLAG_EMAIL               int64
OCCUPATION_TYPE         object
CNT_FAM_MEMBERS        float64
dtype: object

## Data cleaning (application.csv)
- fill blank occupations with "Unemployed"
- inconsistent information: family size < number of children
- change `DAYS_BIRTH` value type to age
- change `DAYS_EMPLOYED` to number of years employed
- assign nan to entries where `DAYS_EMPLOYED` is 365243
- remove duplicated ID, keep the entry that has the older age

In [4]:

app["OCCUPATION_TYPE"] = app["OCCUPATION_TYPE"].fillna("Unemployed")
app.head(5)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Unemployed,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Unemployed,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [5]:
index = app[app["CNT_FAM_MEMBERS"] < app["CNT_CHILDREN"]].index  
app = app.drop(index)

In [6]:
app = app.drop(columns=['FLAG_MOBIL'])

In [7]:
app['AGE'] = (-app['DAYS_BIRTH'] / 365).astype(int)
app = app.drop(columns=['DAYS_BIRTH'])
app.head(5)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_EMPLOYED,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,AGE
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-4542,1,0,0,Unemployed,2.0,32
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-4542,1,0,0,Unemployed,2.0,32
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-1134,0,0,0,Security staff,2.0,58
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-3051,0,1,1,Sales staff,1.0,52
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-3051,0,1,1,Sales staff,1.0,52


In [8]:
app['DAYS_EMPLOYED'] = app['DAYS_EMPLOYED'].replace(365243, np.nan)
app['YEARS_EMPLOYED'] = (-app['DAYS_EMPLOYED'] / 365).astype(float)
app = app.drop(columns=['DAYS_EMPLOYED'])
app['YEARS_EMPLOYED'] = app['YEARS_EMPLOYED'].fillna(0)
app.head(5)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,AGE,YEARS_EMPLOYED
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,1,0,0,Unemployed,2.0,32,12.443836
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,1,0,0,Unemployed,2.0,32,12.443836
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,0,0,0,Security staff,2.0,58,3.106849
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,0,1,1,Sales staff,1.0,52,8.358904
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,0,1,1,Sales staff,1.0,52,8.358904


In [9]:
app.duplicated().sum()

np.int64(0)

In [10]:
app.duplicated(subset='ID').sum() 

np.int64(47)

In [11]:
app.shape

(438516, 17)

In [12]:
app = app.loc[app.groupby('ID')['AGE'].idxmax()].reset_index(drop=True)

In [13]:
app.shape

(438469, 17)

## Data Cleaning (Credit record.csv)
- change months to Months_ago (remove the negative sign)
- choice to decide status: 0,X,C for good credit risk (0) and status >=2 for bad credit risk (1)


In [14]:
cc = pd.read_csv('Data/credit_record.csv')
cc.head(5)

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [15]:
cc['Months_Ago'] = -cc["MONTHS_BALANCE"]

In [16]:
def status_to_binary(status):
    if status in ['0', 'C', 'X']:
        return 0  # good standing
    else:
        return 1  # bad

cc['status_label'] = cc['STATUS'].apply(status_to_binary)

In [17]:
bad_clients = cc[cc['status_label'] == 1]['ID'].unique()
app['TARGET'] = app['ID'].apply(lambda x: 1 if x in bad_clients else 0)
print(app['TARGET'].value_counts())

TARGET
0    434180
1      4289
Name: count, dtype: int64


## feature engineering over time: create aggregate from the history
1. max/ worst status: maximum status value for that client, ignoring X/C
2. mean status: avg repayment behaviour
3. num of bad months with overdue >=1

(not included in dataset yet)

In [ ]:
#agg_cc = cc.groupby('ID').agg(
#    max_status=('Status_Encoded', 'max'),
#    avg_status=('Status_Encoded', 'mean'),
#    num_bad_months=('Status_Encoded', lambda x: sum(x > 1))
#).reset_index()

KeyError: "Column(s) ['Status_Encoded'] do not exist"

## SMOTE

In [ ]:
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split


X = app.drop(columns=['TARGET'])
y = app['TARGET'] 

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)
app.head(5)
categorical_cols = [
    'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
    'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
    'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE'
]

categorical_features = []

for col in categorical_cols:
    idx = X_train.columns.get_loc(col)
    categorical_features.append(idx)

smote_nc = SMOTENC(categorical_features=categorical_features, random_state=42)
X_train_new, y_train_new = smote_nc.fit_resample(X_train, y_train)

In [25]:
print("Before SMOTENC:")
print(y_train.value_counts())

# After SMOTENC
print("\nAfter SMOTENC:")
print(y_train_new.value_counts())

Before SMOTENC:
TARGET
0    347344
1      3431
Name: count, dtype: int64

After SMOTENC:
TARGET
0    347344
1    347344
Name: count, dtype: int64
